In [1]:
!pip install --upgrade azure-cognitiveservices-vision-contentmoderator

  Found existing installation: azure-cognitiveservices-vision-contentmoderator 0.1.0
    Uninstalling azure-cognitiveservices-vision-contentmoderator-0.1.0:
      Successfully uninstalled azure-cognitiveservices-vision-contentmoderator-0.1.0


In [6]:
from azure.cognitiveservices.vision.contentmoderator import ContentModeratorClient
import azure.cognitiveservices.vision.contentmoderator.models
from msrest.authentication import CognitiveServicesCredentials
from pprint import pprint

In [3]:
key = ""
endpoint = ""

In [4]:
client = ContentModeratorClient(endpoint, CognitiveServicesCredentials(key))

### Text Moderation

In [10]:
with open("./Text Moderation.txt", "rb") as text_file:
    text_moderation = client.text_moderation.screen_text(text_content_type="text/plain", text_content=text_file, language="eng", autocorrect=True, pii=True)
    
    pprint(text_moderation.as_dict())

{'auto_corrected_text': 'This food is crap. \n'
                        '\n'
                        'My address is 1 Main St. New York, NY 10001 and my '
                        'phone number is 8645551111',
 'language': 'eng',
 'normalized_text': ' foood  crap. \n'
                    '\n'
                    ' address  1 Main St. New York, NY 10001   phone number  '
                    '8645551111',
 'original_text': 'This foood is crap. \n'
                  '\n'
                  'My address is 1 Main St. New York, NY 10001 and my phone '
                  'number is 8645551111',
 'pii': {'address': [{'index': 36, 'text': '1 Main St. New York, NY 10001'}],
         'email': [],
         'ipa': [],
         'phone': [{'country_code': 'US', 'index': 89, 'text': '8645551111'}],
         'ssn': []},
 'status': {'code': 3000, 'description': 'OK'},
 'terms': [{'index': 8, 'list_id': 0, 'original_index': 14, 'term': 'crap'}],
 'tracking_id': 'USE_ibiza_7b25ac66-7789-4f79-8332-7cb05601a3e

In [12]:
term_list = client.list_management_term_lists.create(
    content_type="application/json",
    body={
        "name": "Custom term list",
        "description": "Custom term description"
    }
)

In [13]:
client.list_management_term.add_term(list_id=term_list.id, term="shenanigans", language="eng")

{'ContentId': '545727',
 'AdditionalInfo': None,
 'Status': {'Code': 3000, 'Description': 'OK', 'Exception': None},
 'TrackingId': 'USE_ibiza_7b25ac66-7789-4f79-8332-7cb05601a3e8_ContentModerator.F0_643a8e54-2c47-416c-89df-4b3bb02c9bbc'}

In [16]:
custom_terms = client.list_management_term.get_all_terms(list_id=term_list.id, language="eng")

pprint(custom_terms.as_dict())

{'data': {'language': 'eng',
          'status': {'code': 3000, 'description': 'OK'},
          'terms': [{'term': 'shenanigans'}],
          'tracking_id': 'USE_ibiza_7b25ac66-7789-4f79-8332-7cb05601a3e8_ContentModerator.F0_5faac7eb-f518-4c7e-a630-d45933f86028'},
 'paging': {'limit': 50, 'offset': 0, 'returned': 1, 'total': 1}}


In [20]:
with open("./Custom Text.txt", "rb") as custom_text:
    screen_text = client.text_moderation.screen_text(
        text_content_type="text/plain", 
        text_content=custom_text, 
        language="eng", 
        autocorrect=True, 
        pii=True,
        list_id=term_list.id
    )
    
    pprint(screen_text.as_dict())

{'auto_corrected_text': 'The movie has a lot of shenanigans.',
 'language': 'eng',
 'normalized_text': ' movie   lot  shenanigans.',
 'original_text': 'The movie has a lot of shenanigans.',
 'status': {'code': 3000, 'description': 'OK'},
 'terms': [{'index': 14,
            'list_id': 1254,
            'original_index': 23,
            'term': 'shenanigans'}],
 'tracking_id': 'USE_ibiza_7b25ac66-7789-4f79-8332-7cb05601a3e8_ContentModerator.F0_af89c2f9-8ee9-4929-a542-81ad8c4b0b81'}


### Image Moderation

In [30]:
# Family Photo by Jude Beck on Unsplash - https://unsplash.com/photos/a-nWU0o73r4

with open ("./family.jpg", "rb") as family:
    image_eval = client.image_moderation.evaluate_file_input(family)
    
    pprint(image_eval.as_dict())

{'adult_classification_score': 0.009908339940011501,
 'advanced_info': [],
 'is_image_adult_classified': False,
 'is_image_racy_classified': False,
 'racy_classification_score': 0.01840316504240036,
 'result': False,
 'status': {'code': 3000, 'description': 'OK'},
 'tracking_id': 'USE_ibiza_7b25ac66-7789-4f79-8332-7cb05601a3e8_ContentModerator.F0_c48d8020-5890-45fe-8964-ea7c412536fb'}


In [29]:
# Swimsuit Photo by Amy Humphries on Unsplash - https://unsplash.com/photos/nvRUmQR3DFM

with open ("./swimsuit.jpg", "rb") as swimsuit:
    image_eval = client.image_moderation.evaluate_file_input(swimsuit)
    
    pprint(image_eval.as_dict())

{'adult_classification_score': 0.022371897473931313,
 'advanced_info': [],
 'is_image_adult_classified': False,
 'is_image_racy_classified': False,
 'racy_classification_score': 0.06550969928503036,
 'result': False,
 'status': {'code': 3000, 'description': 'OK'},
 'tracking_id': 'USE_ibiza_7b25ac66-7789-4f79-8332-7cb05601a3e8_ContentModerator.F0_6f447516-d13e-40fc-9282-b6a7754b9ad5'}


In [38]:
# Sign Photo by Charles Deluvio on Unsplash - https://unsplash.com/photos/znz3wt93jac

with open ("./sign.jpg", "rb") as sign:
    image_eval = client.image_moderation.ocr_file_input(language="eng", image_stream=sign)
        
    pprint(image_eval.as_dict())

{'candidates': [],
 'language': 'eng',
 'metadata': [],
 'status': {'code': 3000, 'description': 'OK'},
 'text': 'KALE SUCKS \r\n',
 'tracking_id': 'USE_ibiza_7b25ac66-7789-4f79-8332-7cb05601a3e8_ContentModerator.F0_589567bb-f91e-48a6-b7ee-f4f4984f145e'}


In [42]:
# Lincoln Photo by Library of Congress on Unsplash - https://unsplash.com/photos/i_9hgrmcUGw

with open ("./lincoln.jpg", "rb") as lincoln:
    image_eval = client.image_moderation.find_faces_file_input(language="eng", image_stream=lincoln)
        
    pprint(image_eval.as_dict())

{'advanced_info': [],
 'count': 1,
 'faces': [{'bottom': 341, 'left': 216, 'right': 395, 'top': 162}],
 'result': True,
 'status': {'code': 3000, 'description': 'OK'},
 'tracking_id': 'USE_ibiza_7b25ac66-7789-4f79-8332-7cb05601a3e8_ContentModerator.F0_90ad9956-4732-431a-bdd8-f8a2b4cab2a3'}


### Review

In [43]:
import uuid

In [44]:
team = "demotest803"
image_url = ""